#### BACKTESTING FEB 2025

In [1]:

import datetime
import joblib
import torch
import torch.nn

from LSTM_Architecture import LSTM, LSTM_V3
from pathlib import Path
from Preprocessing_functions import *
from techinical_analysis import * 